In [2]:
from typing import List, Union
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.encoders import jsonable_encoder
from fastapi.responses import JSONResponse
import pymysql
from pydantic import BaseModel
from starlette.middleware.cors import CORSMiddleware
import certifi
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.dataset import DatasetAutoFolds
from SVD_last import recomm_combi, recomm_game_by_surprise, recomm_game_by_surprise_sortbyiid, get_unplayed_surprise

In [3]:
reader = Reader(line_format = 'user item rating', sep=',', rating_scale=(0.5,10))
data_folds = DatasetAutoFolds(ratings_file='./ratings949.csv', reader=reader)
trainset = data_folds.build_full_trainset()
ratings = pd.read_csv("./bgg-19m-reviews.csv", encoding='UTF-8')
data = Dataset.load_from_df(ratings[["user", "ID", "rating"]], reader)
train_set = data.build_full_trainset()
test_set = train_set.build_testset()
algo = SVD(n_factors=50, n_epochs=20, random_state=42)
algo.fit(trainset)
app = FastAPI()
games = pd.read_csv('./games_detailed_info.csv', encoding='UTF-8', usecols=[2,5])
total_games = games['id'].tolist()


In [4]:
def recommend_movie(user_id:str):



    # -----------추천 알고리즘------------
    top_n = 10
    unplayed_games = get_unplayed_surprise(ratings, total_games, user_id)
    top_games_preds = recomm_game_by_surprise(algo, user_id, unplayed_games,games, top_n)
    return {"result": top_games_preds}

In [5]:
recommend_movie('Torsten')

특정Torsten 유저가 플레이한 게임 수: 1460
 추천한 게임 개수: 20172
 전체 게임수:21631
추천게임이름 Gloomhaven: Jaws of the Lion
예측평점: 9.93791514045237

추천게임이름 Agricola (Revised Edition)
예측평점: 9.876973432300186

추천게임이름 Dominion (Second Edition)
예측평점: 9.748953308105461

추천게임이름 Dungeon Universalis
예측평점: 9.738194728914962

추천게임이름 ISS Vanguard
예측평점: 9.706752701517416

추천게임이름 1822CA
예측평점: 9.697560161330207

추천게임이름 S.H.E.O.L.
예측평점: 9.572132249422088

추천게임이름 Primer: The Gamer's Source for Battles from the Age of Reason
예측평점: 9.53368558527791

추천게임이름 Panzers Last Stand: Battles for Budapest, 1945
예측평점: 9.49832087089408

추천게임이름 Global War 1939
예측평점: 9.476218804270179



{'result': [(209418, 9.93791514045237, 'Gloomhaven: Jaws of the Lion'),
  (292099, 9.876973432300186, 'Agricola (Revised Edition)'),
  (113337, 9.748953308105461, 'Dominion (Second Edition)'),
  (200680, 9.738194728914962, 'Dungeon Universalis'),
  (261393, 9.706752701517416, 'ISS Vanguard'),
  (325494, 9.697560161330207, '1822CA'),
  (253607, 9.572132249422088, 'S.H.E.O.L.'),
  (323046,
   9.53368558527791,
   "Primer: The Gamer's Source for Battles from the Age of Reason"),
  (291457, 9.49832087089408, 'Panzers Last Stand: Battles for Budapest, 1945'),
  (148213, 9.476218804270179, 'Global War 1939')]}

In [9]:
def recommend_combination(ids):
    gameset = []
    for id in ids:
        gameset.append(recomm_combi(algo, id, total_games,games,20 ))

    return gameset

In [10]:
recommend_combination(["Krudte","Ecosmith"])

[[{'gameId': '264476', 'rating': '9.933291495741576'},
  {'gameId': '10783', 'rating': '9.648448038618133'},
  {'gameId': '3771', 'rating': '9.542896879079924'},
  {'gameId': '147887', 'rating': '9.539523816804147'},
  {'gameId': '221248', 'rating': '9.523378711204485'},
  {'gameId': '342210', 'rating': '9.493528063189187'},
  {'gameId': '1422', 'rating': '9.47651741250575'},
  {'gameId': '244939', 'rating': '9.466031190724182'},
  {'gameId': '15985', 'rating': '9.462306437719482'},
  {'gameId': '557', 'rating': '9.459633729087157'},
  {'gameId': '148282', 'rating': '9.440579608262167'},
  {'gameId': '1572', 'rating': '9.425464276886537'},
  {'gameId': '166764', 'rating': '9.420411401503863'},
  {'gameId': '163432', 'rating': '9.415511235127333'},
  {'gameId': '3581', 'rating': '9.400164838614833'},
  {'gameId': '171620', 'rating': '9.399926737052082'},
  {'gameId': '1519', 'rating': '9.399792117458416'},
  {'gameId': '165737', 'rating': '9.398429532838028'},
  {'gameId': '4852', 'rati